<a href="https://colab.research.google.com/github/rgclapp007/gp211-class-notebooks/blob/main/least-squares/vesuvio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install "sep_plot @ git+http://zapad.stanford.edu/bob/pySepPlot.git@b56326716dce35e53e66459cae2227ada16ceabb"


In [ ]:
!wget https://raw.githubusercontent.com/rgclapp007/gp211-class-notebooks/main/data/vesuvio.H


# Vesuvio

In [ ]:
import sep_python.modes
io=sep_python.modes.default_io
vec=io.vector_from_storage("./vesuvio.H")

In [ ]:
x=vec.get_nd_array()
print(type(x),x.dtype,vec.get_data_type())

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh','matplotlib')

vec._hyper.axes=vec._hyper.axes[0:2]

amp=io.get_reg_vector(np.absolute(vec.get_nd_array()),hyper=vec.get_hyper())
phase=io.get_reg_vector(np.angle(vec.get_nd_array()),hyper=vec.get_hyper())

print(amp.get_data_type(),vec.min(),vec.max())
import sep_plot
sep_plot.Grey(amp,bpclip=0,epclip=96).image()+sep_plot.Grey(phase).image()


In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
real=io.get_reg_vector(np.real(vec.get_nd_array()),hyper=vec.get_hyper())
imag=io.get_reg_vector(np.imag(vec.get_nd_array()),hyper=vec.get_hyper())

print(real.min(),real.max(),imag.min(),imag.max())
import sep_plot
sep_plot.Grey(real,bpclip=0,epclip=100).image()+sep_plot.Grey(imag).image()

In [ ]:
from genericSolver.pyOperator import Operator as Operator
from sep_python.sep_vector import FloatVector as FloatVector
class igrad2(Operator):

  def __init__(self,mod,dat):

    if not isinstance(mod,FloatVector) or not isinstance(dat,FloatVector):
      raise Exception("model and data must be FloatVectors")
    
    nmod=mod.get_hyper().get_ns()
    ndat=dat.get_hyper().get_ns()
    if len(nmod)!=2 or len(ndat)!=3:
      raise Exception("Unacceptable dimension")
    
    if nmod[0]!=ndat[0] or nmod[1]!=ndat[1] or ndat[2]!=2:
      raise Exception("Model and data size don't work")
    
    super().__init__(mod,dat)

  def forward(self,add,model,data):
    self.checkDomainRange(model,data)
    if not add:
      data.zero()
    d=data.get_nd_array()
    m=model.get_nd_array()

    d[0,:,:-1]+=m[:,1:]-m[:,:-1]
    d[1,:-1,:]+=m[1:,:]-m[:-1,:]

  def adjoint(self,add,model,data):
    self.checkDomainRange(model,data)
    if not add:
      model.zero()

    d=data.get_nd_array()
    m=model.get_nd_array() 

    m[:,1:]+=d[0,:,:-1]
    m[:,:-1]-=d[0,:,:-1]
    m[1:,:]+=d[1,:-1,:]
    m[:-1,:]-=d[1,:-1,:]




In [ ]:
from sep_python.hypercube import Hypercube
def make_data(cdata):
  ns=cdata.get_hyper().get_ns()
  ns.append(2)
  hyper=Hypercube.set_with_ns(ns)
  dat=io.get_reg_vector(hyper,data_format=np.float32)
  dat.zero()
  cin=cdata.get_nd_array()
  d=dat.get_nd_array()
  print(d.shape,hyper.get_ns())

  a=cin[1:-1,1:-1]
  c=cin[1:-1,2:]
  b=cin[2:,1:-1]

  d[0,1:-1,1:-1]=np.imag(np.log(c*np.conjugate(a)))
  d[1,1:-1,1:-1]=np.imag(np.log(b*np.conjugate(a)))
  return dat


data=make_data(vec)


In [ ]:
from genericSolver.pyProblem import ProblemL2Linear
print(data.min(),data.max())

model=io.get_reg_vector(vec.get_hyper(),dataFromat="float32")
lop=igrad2(model,data)
lop.dotTest(verbose=True)



In [ ]:
x=model.clone()
print(data.min(),data.max())

In [ ]:
print(model.arr.max())

In [ ]:
%xmode verbose
from genericSolver.pyLinearSolver import LCGsolver
from genericSolver.pyStopper import BasicStopper 
model.zero()
problem=ProblemL2Linear(model,data,lop)
stopper=BasicStopper(niter=500)
solver=LCGsolver(stopper)
solver.setDefaults(save_obj=True,save_res=True,iter_sampling=5)
problem.data.dot(problem.data)
solver.run(problem,verbose=True)

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
sep_plot.Grey(problem.res,bpclip=5,epclip=95).image()

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
sep_plot.Grey(problem.model,bpclip=0,epclip=100).image()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(solver.obj[0:])

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
sep_plot.Grey(solver.res[0],bpclip=5,epclip=95).image()

In [ ]:
import scipy.signal
import holoviews as hv
hv.extension('bokeh','matplotlib')
real_new=real.clone()
imag_new=imag.clone()
real_new.get_nd_array()[:]=scipy.signal.medfilt2d(real.get_nd_array(), kernel_size=7)
imag_new.get_nd_array()[:]=scipy.signal.medfilt2d(imag.get_nd_array(), kernel_size=7)

sep_plot.Grey(real_new,bpclip=0,epclip=100).image()+sep_plot.Grey(imag_new).image()

In [ ]:

import holoviews as hv
hv.extension('bokeh','matplotlib')
complexD=real_new.get_nd_array()+imag_new.get_nd_array()*1.j
vec_new=io.get_reg_vector(complexD,hyper=vec.get_hyper())
amp_new=io.get_reg_vector(np.absolute(complexD),hyper=vec.get_hyper())
phase_new=io.get_reg_vector(np.angle(complexD),hyper=vec.get_hyper())
phaseD=phase_new.clone()
phaseD.scale_add(phase,sc2=-1.)
sep_plot.Grey(amp_new,bpclip=3,epclip=97).image()+sep_plot.Grey(phase_new).image()


In [ ]:
data_new=make_data(vec_new)
model.zero()
problem=ProblemL2Linear(model,data_new,lop)
stopper=BasicStopper(niter=500)
solver=LCGsolver(stopper)
solver.setDefaults(save_obj=True,save_res=True,iter_sampling=5)
problem.data.dot(problem.data)
solver.run(problem,verbose=True)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(solver.obj[0:])

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
sep_plot.Grey(problem.res,bpclip=0,epclip=100).image()

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
sep_plot.Grey(problem.model,bpclip=0,epclip=100).image()